In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init
from torch.utils.data import DataLoader

In [2]:
rate=0.001
epoch=50
batch_size = 64

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device =='cuda':
    torch.cuda.manual_seed_all(777)

print(device)

cpu


In [6]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
#데이터 전처리 강화


train_data = dsets.FashionMNIST(root='FashionMNIST_data/', train=True, transform=transform, download=True)

test_data = dsets.FashionMNIST(
    root='FashionMNIST_data/', train=False,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ]),
    download=True
)


In [7]:
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=100, shuffle=False)


In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.dropout1 = nn.Dropout(p=0.5)  # Dropout 추가
        self.dropout2 = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(3 * 3 * 128, 625)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625, 10)

        # 초기화
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        torch.nn.init.kaiming_normal_(self.layer1[0].weight, mode='fan_out', nonlinearity='relu')
        torch.nn.init.kaiming_normal_(self.layer2[0].weight, mode='fan_out', nonlinearity='relu')
        torch.nn.init.kaiming_normal_(self.layer3[0].weight, mode='fan_out', nonlinearity='relu')

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)
        out = self.dropout1(out)
        out = self.fc1(out)
        out = self.dropout2(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [12]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5) #학습률 스케줄러 추가

In [13]:
# 초기 설정
target_accuracy = 95.0  # 목표 정확도 95%
best_accuracy = 0.0  # 가장 좋은 정확도를 기록할 변수
patience = 5  # 정확도가 향상되지 않은 에포크 수
epochs_without_improvement = 0  # 정확도가 향상되지 않은 에포크 수

for e in range(epoch):
    model.train()
    avg_cost = 0
    correct = 0
    total = 0
    scheduler.step()

    # Training loop
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()
        outputs = model(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        avg_cost += loss.item() / len(train_loader)

    # Validation loop
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for x, y in test_loader:
            x = x.to(device)
            y = y.to(device)
            outputs = model(x)
            predicted = torch.argmax(outputs, 1)
            correct += (predicted == y).sum().item()
            total += y.size(0)

        accuracy = 100 * correct / total
        print(f"[Epoch {e}] Accuracy: {accuracy:.2f}%")

        # Early Stopping 조건 체크
        if accuracy >= target_accuracy:
            print(f"목표 정확도 {target_accuracy}%에 도달했습니다. 학습을 종료합니다.")
            break  # 정확도 목표에 도달하면 학습 종료

        # 정확도가 향상되지 않으면 patience 만큼 기다린 후 종료
        if accuracy > best_accuracy:
            best_accuracy = accuracy  # best accuracy 업데이트
            epochs_without_improvement = 0  # 향상되었으므로 초기화
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= patience:
            print(f"정확도가 {patience} 에포크 이상 향상되지 않았습니다. 학습을 종료합니다.")
            break  # 정확도가 향상되지 않으면 학습 종료


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:227: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


[Epoch 0] Accuracy: 85.31%
[Epoch 1] Accuracy: 87.24%
[Epoch 2] Accuracy: 88.71%
[Epoch 3] Accuracy: 89.27%
[Epoch 4] Accuracy: 89.98%
[Epoch 5] Accuracy: 89.54%
[Epoch 6] Accuracy: 90.33%
[Epoch 7] Accuracy: 90.67%
[Epoch 8] Accuracy: 90.43%
[Epoch 9] Accuracy: 91.36%
[Epoch 10] Accuracy: 91.86%
[Epoch 11] Accuracy: 91.90%
[Epoch 12] Accuracy: 91.78%
[Epoch 13] Accuracy: 91.87%
[Epoch 14] Accuracy: 92.09%
[Epoch 15] Accuracy: 91.94%
[Epoch 16] Accuracy: 92.11%
[Epoch 17] Accuracy: 92.17%
[Epoch 18] Accuracy: 92.24%
[Epoch 19] Accuracy: 92.64%
[Epoch 20] Accuracy: 92.79%
[Epoch 21] Accuracy: 92.66%
[Epoch 22] Accuracy: 92.82%
[Epoch 23] Accuracy: 92.81%
[Epoch 24] Accuracy: 92.96%
[Epoch 25] Accuracy: 93.01%
[Epoch 26] Accuracy: 92.81%
[Epoch 27] Accuracy: 92.96%
[Epoch 28] Accuracy: 92.88%
[Epoch 29] Accuracy: 92.99%
[Epoch 30] Accuracy: 93.15%


KeyboardInterrupt: 

In [14]:
model.eval()  # 평가 모드 설정
with torch.no_grad():
    correct = 0
    total = 0
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        outputs = model(x)
        predicted = torch.argmax(outputs, 1)
        correct += (predicted == y).sum().item()
        total += y.size(0)
    accuracy = 100 * correct / total
    print("Accuracy:", accuracy)

print("테스트 완료")

Accuracy: 93.08
테스트 완료
